In [1]:
import spacy
import nltk
import pandas as pd

In [2]:
nlp=spacy.load('en_core_web_lg')

In [3]:
df=pd.read_csv('moviereviews.tsv',sep='\t')

### Word Vectors

In [4]:
nlp(u'There is a dog').vector

array([-1.3071    ,  5.3316746 , -4.4923754 , -1.745275  ,  4.117545  ,
        0.06277502, -1.765725  ,  4.0765495 ,  0.9559575 ,  2.0773022 ,
       10.423725  , -0.42385006, -1.7714683 , -0.16087496,  3.5445375 ,
        3.580075  ,  2.006786  ,  1.7220249 ,  1.06923   ,  0.30959508,
        0.16140008,  1.555975  , -3.7113152 ,  0.21937507, -1.824605  ,
       -2.8309498 , -0.5800251 , -1.6252253 , -3.1270752 , -1.0652299 ,
        1.2348949 ,  0.6315675 , -3.2009501 , -1.1612874 , -1.9292749 ,
        1.6606023 ,  0.5280075 ,  1.6455374 ,  6.48235   , -1.8718251 ,
       -1.635585  , -0.762025  ,  3.2665324 ,  1.2128999 , -2.0247526 ,
        2.1686254 ,  4.864995  , -4.42739   , -1.0794876 , -0.06810248,
        1.52633   ,  2.7944026 ,  4.4619274 , -3.0021253 , -0.41062492,
        0.097875  ,  5.7438498 , -1.0619    , -0.388425  ,  0.341425  ,
        1.684905  , -2.1802852 , -0.95015   , -1.2939501 , -4.12195   ,
       -0.06771243, -4.970125  , -5.876175  ,  2.1812174 ,  3.69

In [5]:
doc=nlp(u'lion cat pet')
for i in doc:
    for j in doc:
        print(i.text,j.text,i.similarity(j))

lion lion 1.0
lion cat 0.3854507803916931
lion pet 0.20031584799289703
cat lion 0.3854507803916931
cat cat 1.0
cat pet 0.732966423034668
pet lion 0.20031584799289703
pet cat 0.732966423034668
pet pet 1.0


In [6]:
doc=nlp(u'like love hate')
for i in doc:
    for j in doc:
        print(i.text,j.text,i.similarity(j))

like like 1.0
like love 0.5212638974189758
like hate 0.5065141320228577
love like 0.5212638974189758
love love 1.0
love hate 0.5708349943161011
hate like 0.5065141320228577
hate love 0.5708349943161011
hate hate 1.0


### Vector Norms

In [7]:
nlp(u'lion').vector_norm #L2 Norm :Euclidean Norm  |x|2=sqrt(sum(xi)^2)

55.14573486169834

In [8]:
nlp(u'liob').has_vector

False

In [9]:
from scipy import spatial
cosine_similarity = lambda vec1,vec2 : 1 - spatial.distance.cosine(vec1,vec2)
king=nlp.vocab['king'].vector
man=nlp.vocab['man'].vector
woman=nlp.vocab['woman'].vector
vector_x=king-man+woman


In [10]:
calculated_similarities=[]
for word in nlp.vocab:
    if word.has_vector:
        if word.is_lower:
            if word.is_alpha:
                similarity=cosine_similarity(vector_x,word.vector)
                calculated_similarities.append((word,similarity))
calculated_similarities=sorted(calculated_similarities,key=lambda item: -item[1])

In [11]:
print(i[0].text for i in calculated_similarities[:5])

<generator object <genexpr> at 0x7f603d7a0a00>


In [12]:
calculated_similarities[3][0].text

'where'

In [13]:
df

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...
...,...,...
1995,pos,"i like movies with albert brooks , and i reall..."
1996,pos,it might surprise some to know that joel and e...
1997,pos,the verdict : spine-chilling drama from horror...
1998,pos,i want to correct what i wrote in a former ret...


In [14]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/akm17/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid=SentimentIntensityAnalyzer()

In [16]:
a="This was a GREAT movie :)"
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.277, 'pos': 0.723, 'compound': 0.8331}

In [17]:
df['label'].value_counts()

label
neg    1000
pos    1000
Name: count, dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2000 non-null   object
 1   review  1965 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [19]:
df.dropna(inplace=True)
empty=[]
for i,j,k in df.itertuples():
    if type(k)==str:
        if k.isspace():
            empty.append(i)
df.drop(empty,inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1938 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1938 non-null   object
 1   review  1938 non-null   object
dtypes: object(2)
memory usage: 45.4+ KB


In [22]:
df['scores']=df['review'].apply(lambda review: sid.polarity_scores(review))
df.head()

,label,review,scores
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co..."
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com..."
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com..."
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co..."
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co..."


In [23]:
df['compound']=df['scores'].apply(lambda score1:score1['compound'])
df['label_vader']=df['compound'].apply(lambda c:'pos' if c>=0 else 'neg')
df.head(10)

,label,review,scores,compound,label_vader
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg
5,neg,"to put it bluntly , ed wood would have been pr...","{'neg': 0.123, 'neu': 0.821, 'pos': 0.056, 'co...",-0.9855,neg
6,neg,"synopsis : melissa , a mentally-disturbed woma...","{'neg': 0.087, 'neu': 0.742, 'pos': 0.17, 'com...",0.9871,pos
7,neg,tim robbins and martin lawernce team up in thi...,"{'neg': 0.118, 'neu': 0.709, 'pos': 0.172, 'co...",0.9829,pos
8,neg,"in "" gia "" , angelina jolie plays the titular ...","{'neg': 0.082, 'neu': 0.862, 'pos': 0.056, 'co...",-0.8278,neg
9,neg,"in 1990 , the surprise success an unheralded l...","{'neg': 0.145, 'neu': 0.728, 'pos': 0.127, 'co...",-0.9147,neg


In [24]:
from sklearn.metrics import accuracy_score

In [25]:
accuracy_score(df['label'],df['label_vader'])

0.6357069143446853

In [27]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(df['label'],df['label_vader']))

              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938

